In [1]:
import gurobipy as gp
from gurobipy import GRB,QuadExpr,nlfunc
import scipy.io
import numpy as np
import math

In [2]:
p = np.array([15,9,8,5,8,2,0])
# p = np.array([21,8,5,10,7,2,0])

In [3]:
lambda1 = 50
lambda2 = 4
lambda3 = 1
lambda4 = 2
lambda5 = 3

In [4]:
c = [
    [1, 2, 2, 0, 0, 2, 1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
c = np.array(c)
T = np.array([30,60,120,180,240,300,360])
time_step = 41
patient_type = 7
bed_number = 14


In [5]:
c.shape[0]

7

In [6]:
# decides if the time slot (i',j',t) exists: j'th appointment slot for a
# type i' patient at time t
B = np.zeros([patient_type,2,time_step])
for i in range(c.shape[0]):
    for j in range(c.shape[1]):
        if c[i,j] == 1:
            B[i,0,j] = 1
        elif c[i,j] == 2:
            B[i,0,j] = 1
            B[i,1,j] = 1  

In [7]:
# decides if the binary variable xo exists(binary variable for override policy 1) 
B_xo = np.zeros([patient_type,patient_type,2,time_step])
for i in range(B_xo.shape[0]):
    for j in range(B_xo.shape[1]):
        if i < j:
            B_xo[i,j,:,:] = B[j,:,:]


In [8]:
# decides if the binary variable x exists(binary variable for override
# policy 1 and cases that exactly assigning the appointments to the time slot)
B_x = np.zeros([patient_type,patient_type,2,time_step])
for i in range(B_x.shape[0]):
    for j in range(B_x.shape[1]):
        if i <= j:
            B_x[i,j,:,:] = B[j,:,:]


In [9]:
# information about the y,z,w (binary variables for override policy 2)
# the chair/bed where time slot(i',j',t) is in
chair = np.zeros([patient_type,time_step,2])
# chair for patient type1 30
chair[0, 0, 0] = 1
chair[0, 1, 0] = 2
chair[0, 1, 1] = 3
chair[0, 2, 0] = 1
chair[0, 2, 1] = 4
chair[0, 5, 0] = 3
chair[0, 5, 1] = 4
chair[0, 6, 0] = 8
chair[0, 7, 0] = 3
chair[0, 7, 1] = 4
chair[0, 8, 0] = 1
chair[0, 9, 0] = 3
chair[0, 10, 0] = 1
chair[0, 10, 1] = 5
chair[0, 18, 0] = 9
chair[0, 20, 0] = 2
chair[0, 20, 1] = 9
chair[0, 23, 0] = 7
chair[0, 25, 0] = 7
chair[0, 31, 0] = 10
chair[0, 32, 0] = 8
chair[0, 33, 0] = 10
chair[0, 34, 0] = 2

# chair for patient type2 60
chair[1, 4, 0] = 1
chair[1, 4, 1] = 2
chair[1, 8, 0] = 2
chair[1, 8, 1] = 8
chair[1, 9, 0] = 4
chair[1, 12, 0] = 1
chair[1, 19, 0] = 7
chair[1, 30, 0] = 2

# chair for patient type3 120
chair[2, 2, 0] = 5
chair[2, 7, 0] = 10
chair[2, 8, 0] = 11
chair[2, 11, 0] = 3
chair[2, 22, 0] = 2
chair[2, 27, 0] = 2
chair[2, 27, 0] = 6
chair[2, 27, 1] = 7
chair[2, 28, 0] = 13
chair[2, 31, 0] = 3

# chair for patient type4 180
chair[3, 3, 0] = 6
chair[3, 6, 0] = 9
chair[3, 8, 0] = 12
chair[3, 9, 0] = 14
chair[3, 12, 0] = 5
chair[3, 15, 0] = 6
chair[3, 16, 0] = 1
chair[3, 16, 1] = 11
chair[3, 19, 0] = 3
chair[3, 28, 0] = 1
chair[3, 28, 1] = 11

# chair for patient type5 240
chair[4, 3, 0] = 7
chair[4, 15, 0] = 10
chair[4, 20, 0] = 12
chair[4, 21, 0] = 14
chair[4, 22, 0] = 9
chair[4, 24, 0] = 5

# chair for patient type6 300
chair[5, 8, 0] = 13
chair[5, 12, 0] = 8

# chair for patient type7 360
chair[6, 13, 0] = 4

In [10]:
chair_tran = np.transpose(chair, (0, 2, 1))

In [11]:
# decides if the two time slots (i',j',t) and (i",j",t+i'/15) on the same bed exists for
# combining
Y = np.zeros([patient_type,patient_type,2,2,time_step])
Z = np.zeros([patient_type,2,time_step])
W = np.zeros([patient_type,2,time_step])

for i in range(c.shape[0]):
    for j in range(c.shape[1]): # it is t
        for h in range(c.shape[0]): # it is i"
            t = int(j+(T[i]/15))
            if c[i,j] != 0 and c[h,t] != 0 and t <= time_step-1:
                if chair[i,j,0] == chair[h,t,0] and chair[i,j,0] != 0:
                    Y[i,h,0,0,j] = 1
                    Z[i,0,j] = 1
                    W[h,0,t] = 1
                if chair[i,j,0] == chair[h,t,1] and chair[i,j,0] != 0:
                    Y[i,h,0,1,j] = 1
                    Z[i,0,j] = 1
                    W[h,1,t] = 1
                if chair[i,j,1] == chair[h,t,0] and chair[i,j,1] != 0:
                    Y[i,h,1,0,j] = 1
                    Z[i,1,j] = 1
                    W[h,0,t] = 1
                if chair[i,j,1] == chair[h,t,1] and chair[i,j,1] != 0:
                    Y[i,h,1,1,j] = 1
                    Z[i,1,j] = 1
                    W[h,1,t] = 1

In [12]:
j+(T[i]/15)-1

63.0

In [13]:
# decides if the appointment could use override policy 2, being assigned to
# a combined time slot 
B_y = np.zeros([patient_type,patient_type,patient_type,2,2,time_step])
B_z = np.zeros([patient_type,patient_type,2,time_step])
B_w = np.zeros([patient_type,patient_type,2,time_step])

for i in range(B_y.shape[0]): #i
    for j in range(B_y.shape[1]): #i'
        for h in range(B_y.shape[2]): #i"
            if i > j and i > h and T[j]+T[h] >= T[i]:
                B_y[i,j,h,:,:,:] = Y[j,h,:,:,:]
                B_z[i,j,:,:] = Z[j,:,:]
                B_w[i,h,:,:] = W[h,:,:]

In [14]:
# information for override policy 3
# decides if the time slot safisfied override policy exist
V = np.zeros([patient_type,time_step,2])
for i in range(c.shape[0]):
    for j in range(c.shape[1]):
        if c[i,j] == 1:
            V[i,j,0] = 1
        elif c[i,j] == 2:
            V[i,j,0] = 1
            V[i,j,1] = 1

In [15]:
# decides if the apoointment i and i" could use override policy in time
# slot(i',t,j') 
B_v = np.zeros([patient_type,patient_type,patient_type,time_step,2])
for i in range(B_v.shape[0]):
    for j in range(B_v.shape[1]):
        for h in range(B_v.shape[2]):
            if i < h and j < h and T[i] + T[j] <= T[h]:
                B_v[i,j,h,:,:] = V[h,:,:]


In [16]:
# index that variables are zero
zeroIndices_bx = (B_x==0);
zeroIndices_bxo = (B_xo==0);
zeroIndices_by = (B_y==0);
zeroIndices_bv = (B_v==0);
zeroIndices_bz = (B_z==0);
zeroIndices_bw = (B_w==0);

In [17]:
# stochastic
mu = T
cov = np.eye(T.shape[0])
alpha = 0.05 # the probability that the actual total appointment time exceeds the total duration of the time slots
L = np.zeros([bed_number,1]) # the total ength of time slots on bed b
for i in range(patient_type):
    for j in range(2):
        for t in range(time_step):
            bed = int(chair[i,t,j])
            if bed != 0:
                L[bed-1] = L[bed-1] + T[i]

In [229]:
version = "deterministic"
model = gp.Model("model 1")
model.setParam("LogFile",f"model1_{version}/log/p1.log")
x = model.addMVar((patient_type,patient_type,2,time_step), vtype = GRB.BINARY,name = "x")
y = model.addMVar((patient_type,patient_type,patient_type,2,2,time_step),vtype = GRB.BINARY,name = "y")
z = model.addMVar((patient_type,patient_type,2,time_step),vtype = GRB.BINARY,name = "z")
w = model.addMVar((patient_type,patient_type,2,time_step),vtype = GRB.BINARY,name = "w")
v = model.addMVar((patient_type,patient_type,patient_type,time_step,2),vtype = GRB.BINARY, name = "v")
q = model.addMVar((patient_type,1),vtype = GRB.INTEGER,name = "q")
# sqrt_term = model.addMVar(bed_number,vtype = GRB.CONTINUOUS, name = "squareroot")
u = gp.MLinExpr.zeros((patient_type,time_step))
u0 = gp.MLinExpr.zeros((patient_type,time_step))
y_c = gp.MLinExpr.zeros((patient_type,patient_type,2,time_step))
# s = gp.MLinExpr.zeros((patient_type,bed_number))
# s = model.addMVar((patient_type,bed_number),vtype = GRB.INTEGER)
xo_s = gp.MLinExpr.zeros((patient_type,patient_type,2,time_step))

Set parameter LogFile to value "model1_deterministic/log/p1.log"


In [230]:
# expression
xo_s = B_xo * x

In [215]:
# stochastic
s = model.addMVar((patient_type,bed_number),vtype = GRB.INTEGER)
for i in range(patient_type):
    for b in range(bed_number):
        xs = 0
        ys = 0
        vs1 = 0
        vs2 = 0
        for ip in range(patient_type):
            for j in range(2):
                for t in range(time_step):
                    if chair[ip,t,j] == b:
                        xs += x[i,ip,j,t]
                        ys += y[i,ip,:,j,:,t].sum()
                        vs1 += v[i,:,ip,t,j].sum()
                        vs2 += v[:,i,ip,t,j].sum()
        model.addConstr(s[i,b-1] == xs + ys + vs1 + vs2)

for b in range(bed_number):
    
    lhs = model.addVar(name=f"lhs_{b}")
    model.addConstr(lhs == ((1-alpha)/alpha) * s[:,b] @ cov @ s[:,b])
    
    rhs = model.addVar(name=f"rhs_{b}")
    model.addConstr(rhs == L[b] - mu @ s[:,b] )
    
    model.addConstr(lhs <= rhs*rhs)


In [231]:
# equation 8
for i in range(patient_type):
    for t in range(time_step):
        u[i,t] = x[i,:,:,t].sum() + y[i,:,:,:,:,t].sum() + v[i,:,:,t,:].sum() + v[:,i,:,t,:].sum()

# equation 9
for i in range(patient_type):
    for t in range(time_step):
        u0[i,t] = xo_s[i,:,:,t].sum() + y[i,:,:,:,:,t].sum() + v[i,:,:,t,:].sum() + v[:,i,:,t,:].sum()

# equation 5
for i in range(patient_type):
    for m in range(patient_type):
        for l in range(2):
            for t in range(time_step):
                for j in range(patient_type):
                    if t-T[j]/15 >= 0:
                        if c[j,int(t-T[j]/15)] != 0:
                            for n in range(c[j,int(t-T[j]/15)]):
                                y_c[i,m,l,t] = y_c[i,m,l,t] + y[i,j,m,n,l,int(t-T[j]/15)]

# constraints
# constraint on the number of appointment assigned
for i in range(patient_type):
    model.addConstr(u[i,:].sum() <= p[i])
# remove redundant variable
model.addConstr(x[zeroIndices_bx] == 0)
model.addConstr(y[zeroIndices_by] == 0)
model.addConstr(v[zeroIndices_bv] == 0)
model.addConstr(z[zeroIndices_bz] == 0)
model.addConstr(w[zeroIndices_bw] == 0)

# equation 10
for i in range(patient_type):
    model.addConstr(q[i] >= 0)
    model.addConstr(q[i] >= p[i] - u[i,:].sum())

# equation 1
for i in range(patient_type): #i'
    for j in range(2):
        for t in range(time_step):
            if B[i,j,t] != 0:
                model.addConstr(x[:,i,j,t].sum() + z[:,i,j,t].sum() + w[:,i,j,t].sum() + (1/2 * v[:,:,i,t,j]).sum() <= 1)

# equation 2,3,4
for i in range(patient_type): # i
    for j in range(patient_type): # i'
        for m in range(patient_type): # i"
            for n in range(2): # j'
                for l in range(2): #j"
                    for t in range(time_step):
                        if B_y[i,j,m,n,l,t] != 0:
                            model.addConstr(y[i,j,m,n,l,t] <= z[i,j,n,t])
                            if t+T[j]/15 <= time_step-1:
                                model.addConstr(y[i,j,m,n,l,t] <= w[i,m,l,int(t+T[j]/15)])
                                model.addConstr(y[i,j,m,n,l,t] >= z[i,j,n,t] + w[i,m,l,int(t+T[j]/15)] - 1)

# equation 5
for i in range(patient_type):
    for m in range(patient_type):
        for l in range(2):
            for t in range(time_step):
                if B_w[i,m,l,t] != 0:
                    model.addConstr(y_c[i,m,l,t] <= 1)
# equation 6
for i in range(patient_type):
    for j in range(patient_type):
        for n in range(2):
            for t in range(time_step):
                if B_z[i,j,n,t] != 0:
                    model.addConstr(y[i,j,:,n,:,t].sum() <= 1)

# equation 7
for i in range(patient_type):
    for t in range(time_step):
        for j in range(2):
            if V[i,t,j] != 0:
                model.addConstr(v[:,:,i,t,j].sum() <= 1)


            

In [232]:
model.setObjective(lambda1*q.sum() + lambda2*u0.sum() + lambda3*xo_s.sum() + lambda4*y.sum() + lambda5*v.sum(),GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.1.0 23B2073)

CPU model: Apple M3 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 96213 rows, 96439 columns and 330001 nonzeros
Model fingerprint: 0x8e991376
Variable types: 0 continuous, 96439 integer (96432 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+00]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 2350.0000000
Presolve removed 96028 rows and 95904 columns
Presolve time: 0.02s
Presolved: 185 rows, 535 columns, 2315 nonzeros
Variable types: 0 continuous, 535 integer (529 binary)
Performing another presolve...
Presolve removed 120 rows and 167 columns
Presolve time: 0.00s

Root relaxation: objective 1.600000e+01, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

In [ ]:
x_arr = np.array(x.X)
y_arr = np.array(y.X)
v_arr = np.array(v.X)
z_arr = np.array(z.X)
w_arr = np.array(w.X)
q_arr = np.array(q.X)

In [236]:
#save result
result = {"x_arr":x_arr,"y_arr":y_arr,"v_arr":v_arr,"z_arr":z_arr,"w_arr":w_arr,"q_arr":q_arr,"B_xo":B_xo}
np.save(f"model1_{version}/result/p1.npy",result)

In [204]:
u_arr = np.zeros([patient_type,time_step])
for i in range(patient_type):
    for t in range(time_step):
        u_arr[i,t] = x_arr[i,:,:,t].sum() + y_arr[i,:,:,:,:,t].sum() + v_arr[i,:,:,t,:].sum() + v_arr[:,i,:,t,:].sum()

In [205]:
amount_check = []
for i in range(patient_type):
    amount_check.append(u_arr[i,:].sum())

In [206]:
amount_check

[15.0, 9.0, 8.0, 5.0, 8.0, 2.0, 0.0]

In [207]:
(B_xo * x.X).sum()

2.0

In [74]:
y.X.sum()

1.0

In [75]:
v.X.sum()

0.0

In [153]:
x_arr.sum()

46.0

In [239]:
# find combined timeslots for an appointment
# x_arr = np.array(x.X)

x_nonzero_idx = np.nonzero(x_arr)

x_nonzero_idx = list(zip(x_nonzero_idx[0],x_nonzero_idx[1], x_nonzero_idx[2],x_nonzero_idx[3]))

x_nonzero_idx #(i,i',j,t)

[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 2),
 (0, 0, 0, 5),
 (0, 0, 0, 6),
 (0, 0, 0, 7),
 (0, 0, 0, 8),
 (0, 0, 0, 9),
 (0, 0, 0, 10),
 (0, 0, 0, 20),
 (0, 0, 0, 32),
 (0, 0, 0, 34),
 (0, 0, 1, 1),
 (0, 0, 1, 2),
 (0, 0, 1, 10),
 (1, 1, 0, 4),
 (1, 1, 0, 8),
 (1, 1, 0, 9),
 (1, 1, 0, 12),
 (1, 1, 0, 19),
 (1, 1, 0, 30),
 (1, 1, 1, 4),
 (1, 1, 1, 8),
 (1, 2, 0, 7),
 (2, 2, 0, 2),
 (2, 2, 0, 11),
 (2, 2, 0, 12),
 (2, 2, 0, 22),
 (2, 2, 0, 27),
 (2, 2, 0, 28),
 (2, 2, 0, 31),
 (2, 2, 1, 27),
 (3, 3, 0, 3),
 (3, 3, 0, 6),
 (3, 3, 0, 8),
 (3, 3, 1, 16),
 (3, 3, 1, 28),
 (4, 4, 0, 3),
 (4, 4, 0, 15),
 (4, 4, 0, 20),
 (4, 4, 0, 21),
 (4, 4, 0, 22),
 (4, 4, 0, 24),
 (4, 6, 0, 13),
 (5, 5, 0, 8),
 (5, 5, 0, 12)]

In [228]:
# find combined timeslots for an appointment
# x_arr = np.array(x.X)

y_nonzero_idx = np.nonzero(y_arr)

y_nonzero_idx = list(zip(y_nonzero_idx[0],y_nonzero_idx[1], y_nonzero_idx[2],y_nonzero_idx[3],y_nonzero_idx[4],y_nonzero_idx[5]))

y_nonzero_idx #(i,i',i",j',j",t)

[(4, 3, 3, 0, 0, 16)]

In [66]:
# find combined timeslots for an appointment
# x_arr = np.array(x.X)

v_nonzero_idx = np.nonzero(v_arr)

v_nonzero_idx = list(zip(v_nonzero_idx[0],v_nonzero_idx[1], v_nonzero_idx[2],v_nonzero_idx[3],v_nonzero_idx[4]))

v_nonzero_idx #(i,i',i",j',t)


[]

In [246]:
#deterministic
loaded_result = np.load('model1_stochastic/result/p1.npy', allow_pickle=True).item()

In [247]:
x_arr = loaded_result["x_arr"]
y_arr = loaded_result["y_arr"]
v_arr = loaded_result["v_arr"]
z_arr = loaded_result["z_arr"]
w_arr = loaded_result["w_arr"]
q_arr = loaded_result["q_arr"]

In [ ]:
# transfer optimal solution for model 3 first step
f = np.zeros([patient_type,14])
beta = np.zeros([patient_type,14])
theta = np.zeros([14])
for ip in range(patient_type):
    for j in range(2):
        for t in range(time_step):
            if x_arr[:,ip,j,t].sum() != 0 or y_arr[:,ip,:,j,:,t].sum() != 0 or v_arr[:,:,ip,t,j].sum() != 0:
                bed = int(chair[ip,t,j])
                for i in range(patient_type):
                    if x_arr[i,ip,j,t] != 0 or y_arr[i,ip,:,j,:,t].sum() != 0 or v_arr[i,:,ip,t,j].sum() != 0 or v_arr[:,i,ip,t,j].sum() != 0:
                        f[i,bed-1] += 1
                        beta[i,bed-1] = 1
                theta[bed-1] = 1


In [ ]:
# check feasibility in model 3 first step
f >= 0

In [ ]:
theta >= beta

In [ ]:
beta <= f

In [ ]:
phi = np.array(p)
res = []
for k in range(patient_type):
    res.append(f[k,:].sum() <= phi[k])

In [ ]:
res

In [ ]:
leng = []
for b in range(14):
    leng.append(T@f[:,b] <= L[b]*theta[b])

In [ ]:
leng